<a href="https://colab.research.google.com/github/carolvieirav/operalabs/blob/main/freightech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Import

In [1]:
!pip install selenium

     |████████████████████████████████| 958 kB 30.3 MB/s 
     |████████████████████████████████| 138 kB 66.0 MB/s 
     |████████████████████████████████| 356 kB 37.1 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 3.6 MB 45.0 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 37 not upgraded.
Need to get 94.0 MB of archives.
After this operation, 324 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 95.0.4638.69-0ubuntu0.18.04.1 [1,135 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 95.0.4638.69-0ubuntu0.18.04.1 [83.6 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 95.0.4638.69-0ubuntu0.18.0

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select

In [4]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

## class bot

In [6]:
class Bot:
  
  def __init__(self,username,password):
    self.username = username
    self.password = password
    self.bot = webdriver.Chrome('chromedriver',options=options)

  def login(self):
    
    bot = self.bot
    bot.get("https://freightech.ambev.com.br/#/login")
    bot.implicitly_wait(10) # seconds
    email = bot.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-input[1]/div/div/input")
    password = bot.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-input[2]/div/div/input")
    email.clear()
    password.clear()
    email.send_keys(self.username)
    password.send_keys(self.password)
    bot.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-button[1]/button").click()
    print("Logged in Successfuly")

In [7]:
automate = Bot("anne.pereira@operalog.com.br", "55026170") 
automate.login()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


Logged in Successfuly


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


## dashboard

In [8]:
def meus_chamados(vigencia, chamadas_pen, chamadas_resol, chamadas_aber, pen_alcada):

  ''' 
  Extract dashboard's data for pages "distribuição", "apoio" e "T1" and store 
  on a pandas dataframe.
  '''

  paths = [vigencia, chamadas_aber, chamadas_resol, chamadas_pen, pen_alcada]

  data = []
  for path in paths:
    automate.bot.implicitly_wait(10) # seconds
    elements = automate.bot.find_elements_by_xpath(path)
    
    for elem in elements:
      automate.bot.implicitly_wait(10) # seconds
      data.append(elem.text)
  
  # cleaning data with replace
  data = [item.replace("\n", " ") for item in data]
  data[0] = data[0].replace('Vigência RESTAM ', '')
  data[1] = data[1].replace('Chamados em aberto ','')
  data[1] = data[1].replace(' Chamados', '')
  data[2] = data[2].replace('Chamados resolvidos ','')
  data[2] = data[2].replace(' Chamados', '')
  data[3] = data[3].replace('Chamados pendentes ','')
  data[3] = data[3].replace(' CHAMADOS', '')
  data[3] = data[3].replace(' APROVAR', '')
  data[4] = data[4].replace('Pendentes por alçada', '')
  data[4] = data[4].replace(' CHAMADOS', '')

  # transform data in pandas dataframe
  data = [pd.Series(item) for item in data]
  df = pd.DataFrame(data)
  df = df.T
  df = df.rename(columns={0:'Vigência', 1:"Chamados em aberto", 2:'Chamados resolvidos', 3:'Chamados pendentes', 4: 'Pendentes por alçada'}, inplace = False)
  
  return df

In [9]:
# *bug* tags de distribuição retornam apenas a primeira data

vigencia_dist =  '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[1]/wac-box/div'
chamadas_aber_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[2]/wac-box/div'
chamadas_resol_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[3]/wac-box/div'
chamadas_pen_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[1]/wac-box/div'
pen_alcada_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[2]/wac-box/div'

dist = meus_chamados(vigencia_dist, chamadas_pen_dist, chamadas_resol_dist, chamadas_aber_dist, pen_alcada_dist)
dist

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  del sys.path[0]


,Vigência,Chamados em aberto,Chamados resolvidos,Chamados pendentes,Pendentes por alçada
0,5 DIAS,0 CHAMADOS ROTA 0 AS 0,30 CHAMADOS ROTA 30 AS 0,ROTA 0 AS 0,GOD 0 CAMPO REGIONAL 0 AC PRODUTIVIDADE 0 AC ...


In [10]:
# *bug* tags do apoio retornam dados da aba distibuição

vigencia_apoio =  '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[1]/wac-box/div'
chamadas_aber_apoio = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[2]/wac-box/div'
chamadas_resol_apoio = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[3]/wac-box/div'
chamadas_pen_apoio = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[1]/wac-box/div'
pen_alcada_apoio = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[2]/wac-box/div'

meus_chamados(vigencia_apoio, chamadas_pen_apoio, chamadas_resol_apoio, chamadas_aber_apoio, pen_alcada_apoio)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  del sys.path[0]


,Vigência,Chamados em aberto,Chamados resolvidos,Chamados pendentes,Pendentes por alçada
0,5 DIAS,0 CHAMADOS ROTA 0 AS 0,30 CHAMADOS ROTA 30 AS 0,ROTA 0 AS 0,GOD 0 CAMPO REGIONAL 0 AC PRODUTIVIDADE 0 AC ...


In [11]:
vigencia_t1 =  '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[1]/wac-box/div'
chamadas_aber_t1 = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[2]/wac-box/div'
chamadas_resol_t1 = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[3]/wac-box/div'
chamadas_pen_t1 = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[1]/wac-box/div'
pen_alcada_t1 = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[2]/wac-box/div'

meus_chamados(vigencia_t1, chamadas_pen_t1, chamadas_resol_t1, chamadas_aber_t1, pen_alcada_t1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  del sys.path[0]


,Vigência,Chamados em aberto,Chamados resolvidos,Chamados pendentes,Pendentes por alçada
0,5 DIAS,0 CHAMADOS ROTA 0 AS 0,30 CHAMADOS ROTA 30 AS 0,ROTA 0 AS 0,GOD 0 CAMPO REGIONAL 0 AC PRODUTIVIDADE 0 AC ...


Resolver: Apesar de todas as abas possuírem as mesmas tags e nomes de classe para designar dados do dashboard, iterar pelo xpath de cada objeto retorna apenas os dados da primeira aba com a primeira data do drop down menu de vigência.

## meus chamados

Construindo uma função que extraia a tabela do "meus chamados" através das tags de table. "tr" para rows e td para columns.

In [45]:
automate.bot.implicitly_wait(10) # seconds
automate.bot.get("https://freightech.ambev.com.br/#/meus-chamados")

In [58]:
row = automate.bot.find_elements_by_xpath('/html/body/app-root/ft-home/div/main/div/wac-meus-chamados/div[3]/div/ft-meus-chamados-detalhes/table/tbody/tr/td/div')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  """Entry point for launching an IPython kernel.


In [60]:
automate.bot.find_elements_by_xpath('/html/body/app-root/ft-home/div/main/div/wac-meus-chamados/div[3]/div/ft-meus-chamados-detalhes/table/tbody/tr/td/div')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  """Entry point for launching an IPython kernel.


[]

## categoria chamados

In [42]:
def categorias_chamados():

  '''
  Extract the table from categoria dos chamados's page and store in a Pandas DataFrame 
  '''

  automate.bot.implicitly_wait(10) # seconds
  automate.bot.get("https://freightech.ambev.com.br/#/categoria-chamados")

  automate.bot.implicitly_wait(10) # seconds
  table = automate.bot.find_elements_by_xpath('/html/body/app-root/ft-home/div/main/div/ft-categoria-chamados/div/div/div/div')

  table_data = []
  for item in table:
    table_data.append(item.text)

  #cleaning data

  table_data = [item.replace("\n", " ") for item in table_data]

  table_data = table_data[0].strip()
  table_data = re.split(" [1-9]. ", table_data)
  teste = [item.split("[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{2}:[0-9]{2}:[0-9]{2}.[0-9]{3}") for item in table_data]
  table_data = [re.split("([0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{2}:[0-9]{2}:[0-9]{2}.[0-9]{3})", item) for item in table_data]

  table_data[0][0] = table_data[0][0].split()
  table_data[0][0].pop(-1)
  table_data[0][0].pop(-2)
  table_data[0] = table_data[0][0]
  
  #dataframe

  table_data = [pd.Series(item) for item in table_data]
  table_categories = pd.DataFrame(table_data)
  table_categories.columns = table_categories.iloc[0]
  table_categories = table_categories.drop(axis=0, index=0)

  table_categories['Usuário'].iloc[-1] = table_categories['Usuário'].iloc[-1].replace(' 1 para 9 de 9 Página 1 de 1', '')

  return table_categories

In [43]:
table_categories = categorias_chamados()
table_categories

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  # This is added back by InteractiveShellApp.init_path()


,Nome,Data,Usuário
1,Sem evidência,2021-05-13T08:58:05.495,Nathalia Fernanda Viana Rodrigues
2,Evidência incorreta,2021-05-13T08:58:18.443,Nathalia Fernanda Viana Rodrigues
3,Sem evidência de validação,2021-05-13T08:58:36.966,Nathalia Fernanda Viana Rodrigues
4,Sem aprovação prévia do GOD ou Regional via Fr...,2021-05-13T08:59:09.611,Nathalia Fernanda Viana Rodrigues
5,Ajuste realizado de forma centralizada,2021-05-13T08:59:28.682,Nathalia Fernanda Viana Rodrigues
6,Divergência entre valor solicitado x valor da ...,2021-05-13T08:59:55.044,Nathalia Fernanda Viana Rodrigues
7,Abertura deve ser feita pelo Sisnec,2021-05-13T09:00:13.265,Nathalia Fernanda Viana Rodrigues
8,Campo incorreto,2021-05-13T09:00:21.916,Nathalia Fernanda Viana Rodrigues
9,Outros,2021-05-14T10:50:23.645,Sergi Blabia Garriga


*   usar regex para selecionar chamados com mais de 1 dígito para dados futuros




## Exportação

Identificando tags com beautifulsoup

* usar autochain do selenium para automatizar cliques e exportar tabelas





In [ ]:
# set implicit wait time
automate.drop.implicitly_wait(10) # seconds

automate.drop.get("https://freightech.ambev.com.br/#/exportacao-dados")
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
modelo = [item.get_text() for item in soup.find_all('p',{'class': "card-fav card-fav-custom no-fav is-selected"})]
modelo

[]